In [10]:
import os
import pandas as pd
import numpy as np

In [11]:
#define directories
cwd = os.getcwd()
pathout = cwd + "/output"
if not os.path.exists(pathout):
    os.mkdir(pathout)

In [ ]:
#load data
dfRNAseq = pd.read_csv('tcga.brca.rsem.csv', low_memory=False)

In [12]:
#QC RNA dataset
dfRNAseq = dfRNAseq.dropna()
dfRNAseq = dfRNAseq.drop_duplicates()
dfRNAseq = dfRNAseq.loc[:, (dfRNAseq != 0).any(axis=0)]

In [13]:
#format RNA dataset
RNAsampletype = dfRNAseq['sample_type']
dfRNAseq.drop(['sample_type'], axis=1, inplace=True)

In [ ]:
#normalize by overall read depth (sum of RSEM counts per sample)
dfnormRNAseq = pd.DataFrame()
for index, row in dfRNAseq.iterrows():
    normarray = []
    rowsum = sum(row)
    for value in row:
        normvalue = value / rowsum
        normarray = np.append(normarray, normvalue)
    tempseries = pd.Series(normarray, name = index)
    dfnormRNAseq = dfnormRNAseq.append(tempseries)

#format and save RSEM normalized data
header = list(dfRNAseq.head(0))
dfnormRNAseq.columns = header
dfnormRNAseq = dfnormRNAseq.assign(sample_type=lambda x: RNAsampletype)
          
filename = pathout + '/RSEMnormalizeddata.txt'
dfnormRNAseq.to_csv(filename, sep='\t', index=True)